In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Carregar modelo e tokenizador
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-llm-7b-chat")
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/deepseek-llm-7b-chat",
    device_map="auto",  # Auto-alocação entre dispositivos
    torch_dtype=torch.float16,  # Usa meia precisão
    low_cpu_mem_usage=True  # Reduz uso de RAM
)

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)

vetorAnimes = FAISS.load_local('C:/Repositorios/RAG-Anime/animes_faiss_index/',embeddings,allow_dangerous_deserialization=True)
vetorEpisodios = FAISS.load_local('C:/Repositorios/RAG-Anime/episodios_faiss_index/',embeddings,allow_dangerous_deserialization=True)
vetorPersonagens = FAISS.load_local('C:/Repositorios/RAG-Anime/personagens_faiss_index/',embeddings,allow_dangerous_deserialization=True)


c:\Anaconda\envs\raganime\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [04:05<00:00, 122.78s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_15412\3714368902.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [2]:
# Função para gerar respostas
def gerar_respostaComVetorAnime(query, k=2):
   
    docAnimes = vetorAnimes.similarity_search(query, k=k)

    context1 = "\n".join([doc.page_content for doc in docAnimes])
    

    # Construir prompt
    prompt = f"""
    Com base nas informações abaixo, responda à pergunta:

    Contextos:
    Contexto geral de animes: {context1}

    Pergunta: {query}

    Resposta (use markdown para destacar títulos):
    """
    
    # Gerar resposta 
    inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=2000, temperature=0.7)
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return resposta.split("Resposta:")[-1].strip()

In [5]:
resposta = gerar_respostaComVetorAnime("kobato")
print(resposta)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
c:\Anaconda\envs\raganime\Lib\site-packages\transformers\generation\utils.py:2105: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Kobato é um anime japonês de fantasia e comédia romântica, produzido pela Studio Deen e exibido originalmente na TV Tokyo entre 2009 e 2010. O anime é baseado na obra literária "Kobato-chan no Ippai Mannequin" (Kobato-chan no Ippai Mannequin) escrita por Yūji Yomogida e ilustrada por Nozomi Tazawa.

    A história segue o enredo de Kobato, uma menina de apenas nove anos que tem um sonho de encontrar o amor perfeito e tornar-se uma "Mãe do Universo". E


In [3]:
# Função para gerar respostas
def gerar_respostaComVetorPersonagens(query, k=2):
   
    docPersonagens = vetorPersonagens.similarity_search(query, k=k)

    context1 = "\n".join([doc.page_content for doc in docPersonagens])
    

    # Construir prompt
    prompt = f"""
    Com base nas informações abaixo, responda à pergunta:

    Contextos:
    Contexto geral de personagens: {context1}

    Pergunta: {query}

    Resposta (use markdown para destacar títulos):
    """
    
    # Gerar resposta 
    inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=2000, temperature=0.7)
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return resposta.split("Resposta:")[-1].strip()

In [6]:
resposta = gerar_respostaComVetorPersonagens("Personagens de Kobato")
print(resposta)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Com base nas informaç�es abaixo, responda à pergunta:

    Contextos:
    Contexto geral de personagens: (Source: Kotonoha)
As Kobato carries out her mission alongside her stuffed toy companion, Ioryogi, she encounters various people troubled by their different situations. From a child struggling with his parents, a high school girl troubled about romance, and everything in between, Kobato's naturally sweet smile and outgoing personality are ready to brighten their day!

[Written by MAL Rewrite]

    Pergunta: Personagens de Kobato

    Resposta (use markdown para destacar títulos):
    
    **Personagens Principais**:
    - Kobato
    - Ioryogi

    **Personagens Secundários**:
    - Hikarizaka
    - Shinobu
    - Saya
    - Otohiko
    - Sanae
    - Tatsuya
    - Kotonoha
    - Koutarou
    - Koutarou's Parents

    **Outros Personagens**:
    - Chizuru
    - Aoi
    - Tetsu
    - Kiyoko
    - Yuzu

    **Personagens Animais**:
    - Ioryogi (Sorrida)
    - Hina (Sorrida)
    - Kana 

In [7]:
# Função para gerar respostas
def gerar_respostaComVetorEpisodios(query, k=2):
   
    docEpis = vetorEpisodios.similarity_search(query, k=k)

    context1 = "\n".join([doc.page_content for doc in docEpis])
    

    # Construir prompt
    prompt = f"""
    Com base nas informações abaixo, responda à pergunta:

    Contextos:
    Contexto geral de episódios: {context1}

    Pergunta: {query}

    Resposta (use markdown para destacar títulos):
    """
    
    # Gerar resposta 
    inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=2000, temperature=0.7)
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return resposta.split("Resposta:")[-1].strip()

In [8]:
resposta = gerar_respostaComVetorEpisodios("Kobato")
print(resposta)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Kobato é uma personagem de anime e mangá japonesa que aparece no jogo "Love Live! School Idol Festival". Ela é uma estudante do ensino médio e também é uma mangáka.
